In [1]:
import os

list_file = sorted(list(os.walk('./file/2.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/2.url/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：10


['https://www.ebay.com/itm/253139036562?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/255922672592?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/235653299500?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/255586148611?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/233621258535?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/332388018897?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/256421824484?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/334041367847?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/256102649999?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/256202118103?_ul=US&_stpos=91710&orig_cvip=true']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        request_url = input_.loc[a, 'Url']
        
        resp = requests.get(request_url,
                            headers=get_header(ua=False),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200 or resp.status_code == 404:
            break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [8]:
resp.status_code == 404

False

In [9]:
import json

dict_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0] + ']}]}')

dict_

{'o': {'l': 1,
  'g': {'enableRumWeb': False,
   'permanentSignal': True,
   'disableUnsafeInline': False,
   'enableATCRedesign': 'false',
   'lazyLoaderType': 'NATIVE',
   'largerImageSize': 'FIXED_15',
   'signedIn': False,
   'epInfo': {'featureFlagDetails': {'enableRumWeb': {'value': False},
     'dwebDeliSSE': {'value': {'NORI_ENABLED': 'true'}, 'dropTag': 'server'},
     'dwebAtfStreamingFeature': {'value': {'NORI_ENABLED': 'false',
       'DELI_ENABLED': 'true'},
      'dropTag': 'server'},
     'prefetchResourceFlag': {'value': {'VINODE_ENABLE_PREFETCH_RESOURCES': 'false'}},
     'dwebAtfSplitFeature': {'value': None},
     'enableOTPFeature': {'value': {'enableOTPFlow': 'false',
       'enableGuestOTPAuthFlow': False},
      'dropTag': 'browser'},
     'enableATCRedesign': {'value': {'enableATCLayerV3': 'false'}},
     'disableUnsafeInline': {'value': False},
     'enableAIImageDisclosure': {'value': False},
     'lazyLoaderType': {'value': False},
     'enableAdBlockerDetect

In [10]:
if input_.loc[a, 'Url'].startswith('https://www.ebay.com/'):
    market, country, language, locale = 'EBAY-US', 'US', 'en', 'en-US'
elif input_.loc[a, 'Url'].startswith('https://www.ebay.de/'):
    market, country, language, locale = 'EBAY-DE', 'DE', 'de', 'de-DE'
elif input_.loc[a, 'Url'].startswith('https://www.ebay.co.uk/'):
    market, country, language, locale = 'EBAY-GB', 'GB', 'en', 'en-GB'
elif input_.loc[a, 'Url'].startswith('https://www.ebay.com.au/'):
    market, country, language, locale = 'EBAY-AU', 'AU', 'en', 'en-AU'
elif input_.loc[a, 'Url'].startswith('https://www.ebay.ca/'):
    market, country, language, locale = 'EBAY-ENCA', 'CA', 'en', 'en-CA'

print(market, country, language, locale)

dict_['o']['g']['marketplaceId'].strip() != market or dict_['o']['g']['country'].strip() != country or dict_['o']['g']['language'].strip() != language or dict_['o']['g']['locale'].strip() != locale or dict_['o']['g']['itemId'].strip() != input_.loc[a, 'Item Number']

EBAY-US US en en-US


False

In [11]:
index_model = -1
for i in range(len(dict_['o']['w'])):
    if 'model' in dict_['o']['w'][i][2]:
        index_model = i
        break

index_model

0

In [12]:
index_model == -1

False

In [13]:
dict_compatibility = dict_['o']['w'][index_model][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['pages'][-1] if 'COMPATIBILITY_TABLE' in dict_['o']['w'][index_model][2]['model']['modules'] else {}
dict_compatibility = {'api': dict_compatibility['action']['URL'],
                      'page': int(dict_compatibility['text'].strip()),
                      'data': {'scopedContext': {'catalogDetails': {'categoryId': dict_compatibility['action']['params']['categoryId'],
                                                                    'itemId': dict_compatibility['action']['params']['itemId'],
                                                                    'marketplaceId': dict_compatibility['action']['params']['marketplaceId'],
                                                                    'sellerName': dict_compatibility['action']['params']['sellerName']}}}} if dict_compatibility else {}

dict_compatibility

{'api': 'https://www.ebay.com/g/api/finders',
 'page': 7,
 'data': {'scopedContext': {'catalogDetails': {'categoryId': '33543',
    'itemId': '253139036562',
    'marketplaceId': 'EBAY-US',
    'sellerName': 'scitoo-autoparts'}}}}

In [14]:
dict_item, dict_count = dict_['o']['w'][index_model][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems'] if 'ABOUT_THIS_ITEM' in dict_['o']['w'][index_model][2]['model']['modules'] else {}, {}
for value in dict_item.values():
    key = value['labels'][0]['textSpans'][0]['text'].strip()
    if key in dict_count:
        dict_count[key] += 1
    else:
        dict_count[key] = 1

dict_specific, dict_no = {}, {}
for i, value in enumerate(dict_item.values()):
    key = value['labels'][0]['textSpans'][0]['text'].strip()
    if dict_count[key] == 1:
        dict_specific[str(i)] = {key: value['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in value['values'][0] else value['values'][0]['textSpans'][0]['text'].strip()}
    else:
        if key in dict_no:
            dict_no[key] += 1
        else:
            dict_no[key] = 1
        dict_specific[str(i)] = {f'''{key}-{dict_no[key]}''': value['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in value['values'][0] else value['values'][0]['textSpans'][0]['text'].strip()}

dict_specific

{'0': {'Condition': "New: A brand-new, unused, unopened, undamaged item in its original packaging (where packaging is applicable). Packaging should be the same as what is found in a retail store, unless the item was packaged by the manufacturer in non-retail packaging, such as an unprinted box or plastic bag. See the seller's listing for full details."},
 '1': {'Brand': 'SCITOO'},
 '2': {'Manufacturer Warranty': '1 Year'},
 '3': {'UPC': '611040278986'},
 '4': {'Manufacturer Part Number': 'CB104TE013S14'},
 '5': {'Interchange Part Number': 'CO 29001C,5005440AA,5005440AA,'},
 '6': {'Other Part Number': '5005440AC,5005441,5005441AC,5005441AD'},
 '7': {'Compressor type': '10S20H'},
 '8': {'Voltage': '12V'},
 '9': {'Fitment Type': 'Direct Replacement'},
 '10': {'Type': 'A/C Compressor'},
 '11': {'Kit Parts Included': '1 x Air Conditioner Compressor'},
 '12': {'Pulley Groove Quantity': '6'},
 '13': {'Pulley Diameter': '130'}}

In [15]:
description_url = dict_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['dataItems'][0]['action']['URL'].strip() if 'dataItems' in dict_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0] else dict_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()

description_url

'https://vi.vipr.ebaydesc.com/itmdesc/253139036562?t=1722937352000&category=33543&seller=scitoo-autoparts&excSoj=1&ver=0&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1'

In [16]:
title = dict_['o']['w'][index_model][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

title

'A/C Compressor Fits 2001 2002 2003-2007 Dodge Chrysler Town &amp; Country 3.3L 3.8L'

In [17]:
subtitle = dict_['o']['w'][index_model][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip() if 'subTitle' in dict_['o']['w'][index_model][2]['model']['modules']['TITLE'] else ''

subtitle

''

In [18]:
price = dict_['o']['w'][index_model][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip() if 'binModel' in dict_['o']['w'][index_model][2]['model']['modules']['BUY_BOX'] else dict_['o']['w'][0][2]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()

price

'US $106.01'

In [19]:
dict_quantity, list_quantity = {}, dict_['o']['w'][index_model][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans'] if 'QUANTITY' in dict_['o']['w'][index_model][2]['model']['modules'] else []
for i, quantity in enumerate(list_quantity):
    dict_quantity[str(i)] = quantity['text'].strip()

dict_quantity

{'0': 'Last one', '1': '61 sold'}

In [20]:
dict_src = {}
list_media = dict_['o']['w'][index_model][2]['model']['modules']['PICTURE']['mediaList'] if 'mediaList' in dict_['o']['w'][index_model][2]['model']['modules']['PICTURE'] else []
for i, media in enumerate(list_media):
    dict_src[str(i)] = media['image']['originalImg']['URL'].strip()

dict_src

{'0': 'https://i.ebayimg.com/images/g/g~MAAOSwEIBjXPwk/s-l500.jpg',
 '1': 'https://i.ebayimg.com/images/g/0TgAAOSwBdhjXPwl/s-l500.jpg',
 '2': 'https://i.ebayimg.com/images/g/6QIAAOSwuSRjXPwm/s-l500.jpg',
 '3': 'https://i.ebayimg.com/images/g/8F8AAOSwKX1jXPwn/s-l500.jpg',
 '4': 'https://i.ebayimg.com/images/g/YLMAAOSwD1JjXPwo/s-l500.jpg',
 '5': 'https://i.ebayimg.com/images/g/VW4AAOSwAUtjXPwp/s-l500.jpg',
 '6': 'https://i.ebayimg.com/images/g/C2QAAOSws~ZjXPwq/s-l500.jpg'}

In [21]:
df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Item Number': input_.loc[a, 'Item Number'],
                         'Market': country,
                         'Title': title,
                         'Price': price,
                         'Picture': '',
                         'Url': input_.loc[a, 'Url'],
                         'Json_Src': json.dumps(dict_src),
                         'Description Url': description_url,
                         'Subtitle': subtitle,
                         'Json_Quantity': json.dumps(dict_quantity),
                         'Json_Compatibility': json.dumps(dict_compatibility),
                         'Json_Specific': json.dumps(dict_specific)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Item Number,Market,Title,Price,Picture,Url,Json_Src,Description Url,Subtitle,Json_Quantity,Json_Compatibility,Json_Specific
0,1,253139036562,US,A/C Compressor Fits 2001 2002 2003-2007 Dodge ...,US $106.01,,https://www.ebay.com/itm/253139036562?_ul=US&_...,"{""0"": ""https://i.ebayimg.com/images/g/g~MAAOSw...",https://vi.vipr.ebaydesc.com/itmdesc/253139036...,,"{""0"": ""Last one"", ""1"": ""61 sold""}","{""api"": ""https://www.ebay.com/g/api/finders"", ...","{""0"": {""Condition"": ""New: A brand-new, unused,..."


In [22]:
crawler_status = 'ok'

crawler_status

'ok'

In [23]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [24]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,No,Item Number,Market,Title,Price,Picture,Url,Json_Src,Description Url,Subtitle,Json_Quantity,Json_Compatibility,Json_Specific
0,1,253139036562,US,A/C Compressor Fits 2001 2002 2003-2007 Dodge ...,US $106.01,,https://www.ebay.com/itm/253139036562?_ul=US&_...,"{""0"": ""https://i.ebayimg.com/images/g/g~MAAOSw...",https://vi.vipr.ebaydesc.com/itmdesc/253139036...,,"{""0"": ""Last one"", ""1"": ""61 sold""}","{""api"": ""https://www.ebay.com/g/api/finders"", ...","{""0"": {""Condition"": ""New: A brand-new, unused,..."


In [25]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
